In [1]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
import numpy as np

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder

Using Theano backend.


In [2]:
#Load dataset

#train data

xin= np.load("./X_train.npy")
yin= np.reshape(np.load("./y_train.npy"), (-1,1))
xout= np.load("./X_test.npy")

img_xin = np.reshape(xin, (966,1,50,37), order='C')
img_xout = np.reshape(xout, (322,1,50,37), order='C')
print(xin.shape)
print(xout.shape)
print(img_xin.shape)
print(img_xout.shape)
enc = OneHotEncoder()
enc.fit(yin)
yin = enc.transform(yin).toarray()

(966, 1850)
(322, 1850)
(966, 1, 50, 37)
(322, 1, 50, 37)


In [3]:
# Helper functions
def createSubmission(filename, y):
    fo = open( filename , 'w' )
    fo.write("ImageId,PredictedClass\n")
    for i in range(y.shape[0]):
        fo.write(str(i)+","+str(y[i])+"\n")
    fo.close()

def nnPredict(model,x, y):
    ypredict= model.predict(x)
    score= f1_score(y , ypredict, average='macro');
    return score

In [ ]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(1, 50, 37), activation='relu'))
model.add(Convolution2D(32, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7))
model.add(Activation('softmax'))

adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['fbeta_score'])

model.fit(img_xin, yin, nb_epoch=200, batch_size=64, validation_split=0.5)

Train on 483 samples, validate on 483 samples
Epoch 1/200
483/483 [==============================] - 12s - loss: 12.0377 - fbeta_score: 0.1555 - val_loss: 8.2673 - val_fbeta_score: 0.4037
Epoch 2/200
483/483 [==============================] - 11s - loss: 10.8247 - fbeta_score: 0.2388 - val_loss: 9.0976 - val_fbeta_score: 0.4017
Epoch 3/200
483/483 [==============================] - 10s - loss: 10.0659 - fbeta_score: 0.2798 - val_loss: 9.5278 - val_fbeta_score: 0.4017
Epoch 4/200
483/483 [==============================] - 10s - loss: 9.8158 - fbeta_score: 0.3171 - val_loss: 9.6288 - val_fbeta_score: 0.4017
Epoch 5/200
483/483 [==============================] - 10s - loss: 9.6588 - fbeta_score: 0.3499 - val_loss: 9.6425 - val_fbeta_score: 0.4017
Epoch 6/200
483/483 [==============================] - 11s - loss: 9.2865 - fbeta_score: 0.3793 - val_loss: 9.6442 - val_fbeta_score: 0.4017
Epoch 7/200
 64/483 [==>...........................] - ETA: 6s - loss: 10.2902 - fbeta_score: 0.3125

In [ ]:
model.fit(img_xin, yin, nb_epoch=100, batch_size=64)

In [ ]:
y_pred = model.predict(img_xout, batch_size=64, verbose=1)


In [ ]:
# Convert from one-hot encoding to integer labels
y_predicted = np.argmax(y_pred, axis=1)

In [ ]:
createSubmission("submission_cnn_10_31.csv", y_predicted)